# Data Extraction: Web Scraping 

## I. Dataset: Team Rating 

In [81]:
import requests
from bs4 import BeautifulSoup
import lxml.html as lh
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from re import sub
from decimal import Decimal
import numpy as np
%matplotlib inline

In [82]:
url = 'https://www.basketball-reference.com/leagues/NBA_2019_ratings.html'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)

In [83]:
#parse the first row of the table
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[1]:
    i+=1
    name=t.text_content()
#     print ('%d:"%s"'%(i,name))
    
    col.append((name,[]))

In [84]:
#Since out first two rows are the header, data is stored on the third row onwards
for j in range(2,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 15, the //tr data is not from our table 
    if len(T)!=15:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [85]:
Dict={title:column for (title,column) in col}
team_rate=pd.DataFrame(Dict)

In [86]:
#we are using only the adjusted ratings
team_rate=team_rate.drop(["Rk","Conf","Div",'W','L',"MOV","ORtg","DRtg","NRtg",'ORtg/A','DRtg/A'], axis=1)

In [87]:
col=['W/L%','NRtg/A']
team_rate[col]=team_rate[col].apply(pd.to_numeric)

In [88]:
#export to csv file
team_rate.to_csv('team_rate.csv')

## II. Dataset: NBA Player Salaries

In [8]:
url_list=[]

for page in range(1,16):
    url='http://www.espn.com/nba/salaries/_/year/2019/page/'+ str(page) + '/seasontype/4'
    url_list.append(url)

In [9]:
df_list = []
for url in url_list:
    df_list.append(pd.read_html(url)[0]) 

In [10]:
salary = pd.concat(df_list)
salary.columns = ['Rank', 'Name','Team', 'Salary']

In [11]:
salary = salary[~(salary['Rank'] == 'RK')]

In [13]:
salary['Name'] = salary['Name'].apply(lambda name: name.split(',')[0])

In [14]:
salary=salary.drop("Rank",axis=1)

In [15]:
#convert salary from string to numerical number
salary.Salary=salary.Salary.apply(lambda m: Decimal(sub(r'[^\d.]', '', m)))

In [16]:
salary.Salary=salary.Salary.astype(int)

In [ ]:
salary.to_csv('salary.csv')

## III. Dataset: Team Abbreviation

In [17]:
url = 'https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Basketball_Association/National_Basketball_Association_team_abbreviations'
team = pd.read_html(url)[0]

In [18]:
team=team[1:]

In [19]:
team=team.rename(columns={ team.columns[0]: "Tm" ,team.columns[1]:'Team'})

In [ ]:
team.to_csv('team.csv')

## III. Dataset: Players WinShare Statistics 

In [47]:
url='https://www.basketball-reference.com/play-index/psl_finder.cgi?request=1&match=single&type=advanced&per_minute_base=36&per_poss_base=100&lg_id=NBA&is_playoffs=N&year_min=2019&year_max=2019&franch_id=&season_start=1&season_end=-1&age_min=0&age_max=99&shoot_hand=&height_min=0&height_max=99&birth_country_is=Y&birth_country=&birth_state=&college_id=&draft_year=&is_active=&debut_yr_nba_start=&debut_yr_nba_end=&is_hof=&is_as=&as_comp=gt&as_val=0&award=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&qual=&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&c5stat=&c5comp=&c6mult=&c6stat=&order_by=ws&order_by_asc=&offset=0'


In [48]:
url_list=[]

for page in range(0,600,100):
    url='https://www.basketball-reference.com/play-index/psl_finder.cgi?request=1&match=single&type=advanced&per_minute_base=36&per_poss_base=100&lg_id=NBA&is_playoffs=N&year_min=2019&year_max=2019&franch_id=&season_start=1&season_end=-1&age_min=0&age_max=99&shoot_hand=&height_min=0&height_max=99&birth_country_is=Y&birth_country=&birth_state=&college_id=&draft_year=&is_active=&debut_yr_nba_start=&debut_yr_nba_end=&is_hof=&is_as=&as_comp=gt&as_val=0&award=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&qual=&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&c5stat=&c5comp=&c6mult=&c6stat=&order_by=ws&order_by_asc=&offset='+str(page)
    url_list.append(url)

In [49]:
table=[]
for url in url_list:
    url=pd.read_html(url)[0]
    table.append(url)

In [50]:
advanced_stat=pd.concat(table)

In [51]:
#drop the multi-index level
advanced_stat.columns=advanced_stat.columns.droplevel(0)

In [52]:
#change the index
advanced_stat=advanced_stat.reset_index(drop=True)

In [53]:
title=advanced_stat[advanced_stat['Rk']=='Rk'].index
advanced_stat.drop(title,inplace=True)

In [54]:
advanced_stat=advanced_stat.rename(columns={'Player':'Name'})
name_col=['Rk','Season','Lg','GS']
advanced_stat=advanced_stat.drop(name_col,axis=1)

In [55]:
advanced_stat.isnull().sum()

Name     0
Age      0
Tm       0
WS       0
G        0
MP       0
PER      0
3PAr     4
FTr      4
ORB%     0
DRB%     0
TRB%     0
AST%     0
STL%     0
BLK%     0
TOV%     4
USG%     0
ORtg     4
DRtg     0
OWS      0
DWS      0
WS/48    0
OBPM     0
DBPM     0
BPM      0
VORP     0
dtype: int64

In [56]:
#check where the null values are located
advanced_stat[advanced_stat['3PAr'].isnull()].loc[:,['Name','G','MP','3PAr','FTr','TOV%','ORtg','USG%']]
#it seems like that these people didn't play that much at the field 

,Name,G,MP,3PAr,FTr,TOV%,ORtg,USG%
473,John Holland,1,1,NaN,NaN,NaN,NaN,0.0
476,George King,1,6,NaN,NaN,NaN,NaN,0.0
494,Kobi Simmons,1,2,NaN,NaN,NaN,NaN,0.0
500,Tyler Ulis,1,1,NaN,NaN,NaN,NaN,0.0


In [63]:
#drop the rows where they have nan values
advanced_stat=advanced_stat[advanced_stat['3PAr'].notnull()]

In [65]:
col_list=['Age','WS', 'G', 'GS', 'MP', 'PER', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg',
       'DRtg', 'OWS', 'DWS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']
advanced_stat[col_list] = advanced_stat[col_list].apply(pd.to_numeric)

In [66]:
advanced_stat.shape

(526, 26)

In [61]:
advanced_stat.to_csv('advanced_stat.csv')

## IV. Dataset: Players Basic Statistics

In [67]:
url = 'https://www.basketball-reference.com/leagues/NBA_2019_per_game.html'

In [68]:
player_stat=pd.read_html(url)[0]

In [69]:
title=player_stat[player_stat['Rk']=='Rk'].index
title

Int64Index([ 22,  49,  70,  97, 132, 161, 186, 217, 244, 269, 297, 324, 349,
            382, 411, 438, 468, 498, 527, 554, 579, 606, 642, 671, 694, 715],
           dtype='int64')

In [70]:
player_stat.drop(title,inplace=True)

In [71]:
player_stat.isnull().sum()

Rk         0
Player     0
Pos        0
Age        0
Tm         0
G          0
GS         0
MP         0
FG         0
FGA        0
FG%        6
3P         0
3PA        0
3P%       47
2P         0
2PA        0
2P%       15
eFG%       6
FT         0
FTA        0
FT%       43
ORB        0
DRB        0
TRB        0
AST        0
STL        0
BLK        0
TOV        0
PF         0
PTS        0
dtype: int64

In [37]:
#FT% is calculated by FT/FTA which means when FT is 0, FT% is null
#so we can fill those cells with 0
player_stat=player_stat.fillna(0)

In [72]:
col_list=['Age','G','GS','MP','3P%', '2P%','eFG%', 'FT%','TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS']
player_stat[col_list] = player_stat[col_list].apply(pd.to_numeric)

In [73]:
player_stat=player_stat.rename(columns={'Player':'Name'})

In [74]:
player_stat = player_stat.sort_values('G', ascending=False)

In [75]:
player_stat[player_stat["Name"]=="Ivica Zubac"]

,Rk,Name,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
731,530,Ivica Zubac,C,21,TOT,59,37,17.6,3.6,6.4,...,0.802,1.9,4.2,6.1,1.1,0.2,0.9,1.2,2.3,8.9
732,530,Ivica Zubac,C,21,LAL,33,12,15.6,3.4,5.8,...,0.864,1.6,3.3,4.9,0.8,0.1,0.8,1.0,2.2,8.5
733,530,Ivica Zubac,C,21,LAC,26,25,20.2,3.8,7.2,...,0.733,2.3,5.3,7.7,1.5,0.4,0.9,1.4,2.5,9.4


In [76]:
player_stat.shape

(708, 30)

In [77]:
#since some players play for multiple teams, we only consider them as the team they played for the most
player_stat = player_stat.drop_duplicates(subset='Name', keep='first')

In [44]:
player_stat[player_stat["Name"]=="Ivica Zubac"]

,Rk,Name,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
731,530,Ivica Zubac,C,21,TOT,59,37,17.6,3.6,6.4,...,0.802,1.9,4.2,6.1,1.1,0.2,0.9,1.2,2.3,8.9


In [78]:
player_stat.shape
#interesting insight: almost 200 players play for multiple teams

(530, 30)

In [79]:
#columns that we would want to keep
player_stat=player_stat.loc[:,['Name','Pos','Age','GS','eFG%','PF','PTS']]
player_stat.head()

,Name,Pos,Age,GS,eFG%,PF,PTS
310,Justin Holiday,SG,29,77,0.490,2.0,10.5
480,Patty Mills,PG,30,1,0.545,1.6,9.9
330,Joe Ingles,PF,31,82,0.565,2.2,12.1
54,DeAndre' Bembry,SG,24,15,0.487,2.3,8.4
674,P.J. Tucker,PF,33,82,0.535,3.1,7.3


In [80]:
player_stat.to_csv('player_stat.csv')